In [1]:
# Imports
import torch
from dask_jobqueue import SLURMCluster
from distributed import Client
from linear_model import run_experiment

In [2]:
seeds = 50
nsamples = [1000]

In [3]:
cluster = SLURMCluster(
    n_workers=0,
    memory="8GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":10092",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="24:0:0",
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

In [4]:
futures = []
noise_types = ["normal", "gamma", "cauchy", "inversegamma", "rademacher"]

for seed in range(seeds):
    for n in nsamples:
        for noise in noise_types:
            f = client.submit(run_experiment, seed, n, noise_type=noise)
            futures.append(f)

In [5]:
results = client.gather(futures)

In [6]:
client.close()
cluster.close()

2025-06-21 12:25:38,370 - distributed.deploy.adaptive_core - INFO - Adaptive stop


In [7]:
torch.save(f = "linear_model_results.pt", obj = results)